In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install "transformers>4.42.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 72.6 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.169.0
    Uninstalling google-api-python-client-2.169.0:
      Successfully uninstalled google-api-python-client-2.169.0


In [2]:
import json

with open('Data_RandomDate_finetune-Tell.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data_test = data['test']

In [3]:
for item in data_test:
  print(item)
  break

{'messages': [{'role': 'system', 'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-12-27.\n\nCurrent Day: Saturday.'}, {'role': 'user', 'content': 'เพิ่มแจ้งเตือนนัดถกกันเรื่องโปรเจควันที่ 5 เมษา ตอนบ่ายโมงสิบห้า'}, {'role': 'assistant', 'content': "<tool_call>\n{'name': 'add_event_date', 'arguments': {'date': '2026-04-05', 'time': '13:15', 'title': 'ถกกันเรื่องโปรเจค'}}\n</tool_call>"}]}


In [4]:
with open('tools.json', 'r', encoding='utf-8') as f:
    TOOLS = json.load(f)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "TechitoTamani/Qwen3-4B_FinetuneWithMyData-3-AddDayTelling-2"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype="auto",
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/132M [00:00<?, ?B/s]

In [6]:
data_test_user_messages = [
    {
        "messages": [
            message for message in item["messages"]
            if message["role"] in ["user", "system"]
        ]
    }
    for item in data_test
]

In [7]:
data_test_user_messages[16]["messages"]

[{'role': 'system',
  'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-11-13.\n\nCurrent Day: Thursday.'},
 {'role': 'user',
  'content': 'เลื่อนเวลานัดคุยงานหัวข้อ AI Buildersวันอาทิตย์ที่จะถึงนี้ ไปเป็นสี่โมงเย็น 30 นาที'}]

In [8]:
import re
import ast

def convert_to_dict(text):
    match = re.search(r"<tool_call>\n(.*?)\n</tool_call>", text)

    if match:
        tool_dict_str = match.group(1)
        try:
            result = ast.literal_eval(tool_dict_str)
            return(result)
        except Exception as e:
            return({})
    else:
        return({})


# **Put it on together**

In [9]:
def model_answer(messages):
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=False,
      tools=TOOLS,
      enable_thinking=False
  )
  inputs = tokenizer(text, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=512)
  output_text = tokenizer.batch_decode(outputs)[0][len(text):]

  return(output_text)

In [10]:
data_test_user_messages[0]["messages"]

[{'role': 'system',
  'content': 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\nCurrent Date: 2025-12-27.\n\nCurrent Day: Saturday.'},
 {'role': 'user',
  'content': 'เพิ่มแจ้งเตือนนัดถกกันเรื่องโปรเจควันที่ 5 เมษา ตอนบ่ายโมงสิบห้า'}]

In [11]:
print(model_answer(data_test_user_messages[0]["messages"]))

<|im_start|>assistant
<think>

</think>

<tool_call>
{'name': 'add_event_date', 'arguments': {'date': '2026-04-05', 'time': '13:15', 'title': 'ถกกันเรื่องโปรเจค'}}
</tool_call><|im_end|>


# **Predict**

In [12]:
predict = []
finished = 0
import copy

In [13]:
from tqdm import tqdm
import copy

for data_k in tqdm(data_test_user_messages):
    messages = copy.deepcopy(data_k["messages"])
    prediction = model_answer(messages)
    messages.append({"role": "assistant","content":prediction})
    predict.append({"messages": messages})

    finished += 1
    if finished%10 == 0:
      with open('Qwen_3-4b_Finetuned-predict-UpdatedData-4-2.json', 'w', encoding='utf-8') as f:
        json.dump(predict, f, ensure_ascii=False, indent=2)

with open('Qwen_3-4b_Finetuned-predict-UpdatedData-4-2.json', 'w', encoding='utf-8') as f:
  json.dump(predict, f, ensure_ascii=False, indent=2)

100%|██████████| 296/296 [36:56<00:00,  7.49s/it]
